In [7]:
%load_ext autoreload
%autoreload 2

import functools
import tensorflow as tf
from pathlib import Path
from model.data_loader import DL
from model.model import model_fn

LABEL_COL = 3
DATADIR = 'data/conll2003'

# Params
params = {
    'dim_chars': 100,
    'dim': 300,
    'dropout': 0.5,
    'num_oov_buckets': 1,
    'epochs': 25,
    'label_col': 3,
    'batch_size': 20,
    'buffer': 15000,
    'filters': 50,
    'kernel_size': 3,
    'lstm_size': 100,
    'words': str(Path(DATADIR, 'vocab.words.txt')),
    'chars': str(Path(DATADIR, 'vocab.chars.txt')),
    'tags': str(Path(DATADIR, 'vocab.tags.txt')),
    'glove': str(Path(DATADIR, 'glove.npz')),
    'fulldoc': True
}

DL().set_params(params=params)

train_inpf = functools.partial(DL().input_fn, 'train', training=True)
eval_inpf  = functools.partial(DL().input_fn, 'valid')

cfg = tf.estimator.RunConfig(save_checkpoints_secs=120)
estimator = tf.estimator.Estimator(model_fn, 'results/model', cfg, params)
Path(estimator.eval_dir()).mkdir(parents=True, exist_ok=True)

# Early stop if F1 does not increase.
hook = tf.contrib.estimator.stop_if_no_increase_hook(estimator, 'f1', 800, min_steps=8000, run_every_secs=120)

train_spec = tf.estimator.TrainSpec(input_fn=train_inpf, hooks=[hook], max_steps=10000)
eval_spec = tf.estimator.EvalSpec(input_fn=eval_inpf, throttle_secs=120)
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
INFO:tensorflow:Using config: {'_is_chief': True, '_experimental_distribute': None, '_service': None, '_model_dir': 'results/model', '_protocol': None, '_global_id_in_cluster': 0, '_task_type': 'worker', '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9769a03390>, '_master': '', '_device_fn': None, '_keep_checkpoint_max': 5, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_eval_distribute': None, '_train_distribute': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_save_checkpoints_secs': 120, '_task_id': 0, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluat

IndexError: list index out of range

In [ ]:
# Write predictions to file.
def write_predictions(name):
  Path('results/score').mkdir(parents=True, exist_ok=True)
  with Path('results/score/{}.preds.txt'.format(name)).open('wb') as f:
    test_inpf = functools.partial(DL().input_fn, name)
    golds_gen = DL().generator_fn(name)
    preds_gen = estimator.predict(test_inpf)
    for golds, preds in zip(golds_gen, preds_gen):
      ((words, _),(_, _)), tags = golds
      for word, tag, tag_pred in zip(words, tags, preds['tags']):
        f.write(b' '.join([word, tag, tag_pred]) + b'\n')
      f.write(b'\n')

for name in ['train', 'valid', 'test']:
  write_predictions(name)

In [ ]:
!./conlleval < results/score/train.preds.txt
!./conlleval < results/score/valid.preds.txt
!./conlleval < results/score/test.preds.txt

```python
processed 203621 tokens with 23499 phrases; found: 23439 phrases; correct: 22797.
accuracy:  99.58%; precision:  97.26%; recall:  97.01%; FB1:  97.14
              LOC: precision:  98.20%; recall:  98.56%; FB1:  98.38  7166
             MISC: precision:  94.54%; recall:  93.16%; FB1:  93.85  3388
              ORG: precision:  96.91%; recall:  95.33%; FB1:  96.12  6218
              PER: precision:  97.96%; recall:  98.95%; FB1:  98.45  6667
processed 51362 tokens with 5942 phrases; found: 5939 phrases; correct: 5536.
accuracy:  98.86%; precision:  93.21%; recall:  93.17%; FB1:  93.19
              LOC: precision:  95.75%; recall:  96.79%; FB1:  96.26  1857
             MISC: precision:  88.57%; recall:  87.42%; FB1:  87.99  910
              ORG: precision:  91.81%; recall:  86.95%; FB1:  89.31  1270
              PER: precision:  93.90%; recall:  96.96%; FB1:  95.41  1902
processed 46435 tokens with 5648 phrases; found: 5650 phrases; correct: 5088.
accuracy:  97.94%; precision:  90.05%; recall:  90.08%; FB1:  90.07
              LOC: precision:  90.79%; recall:  93.35%; FB1:  92.05  1715
             MISC: precision:  79.22%; recall:  77.64%; FB1:  78.42  688
              ORG: precision:  88.86%; recall:  86.45%; FB1:  87.64  1616
              PER: precision:  95.03%; recall:  95.86%; FB1:  95.44  1631
                            
processed 217662 tokens with 23499 phrases; found: 23486 phrases; correct: 23327.
accuracy:  99.92%; precision:  99.32%; recall:  99.27%; FB1:  99.30
              LOC: precision:  99.52%; recall:  99.58%; FB1:  99.55  7144
             MISC: precision:  98.80%; recall:  98.23%; FB1:  98.51  3418
              ORG: precision:  99.05%; recall:  99.08%; FB1:  99.07  6323
              PER: precision:  99.64%; recall:  99.65%; FB1:  99.64  6601
processed 54612 tokens with 5942 phrases; found: 5944 phrases; correct: 5611.
accuracy:  99.09%; precision:  94.40%; recall:  94.43%; FB1:  94.41
              LOC: precision:  96.02%; recall:  97.06%; FB1:  96.53  1857
             MISC: precision:  90.91%; recall:  87.85%; FB1:  89.35  891
              ORG: precision:  92.16%; recall:  91.20%; FB1:  91.68  1327
              PER: precision:  96.04%; recall:  97.45%; FB1:  96.74  1869
processed 49888 tokens with 5648 phrases; found: 5645 phrases; correct: 5126.
accuracy:  98.19%; precision:  90.81%; recall:  90.76%; FB1:  90.78
              LOC: precision:  91.63%; recall:  93.23%; FB1:  92.42  1697
             MISC: precision:  82.07%; recall:  78.92%; FB1:  80.46  675
              ORG: precision:  87.66%; recall:  88.56%; FB1:  88.11  1678
              PER: precision:  96.93%; recall:  95.61%; FB1:  96.26  1595
                            
processed 217662 tokens with 23499 phrases; found: 23492 phrases; correct: 23441.
accuracy:  99.97%; precision:  99.78%; recall:  99.75%; FB1:  99.77
              LOC: precision:  99.83%; recall:  99.86%; FB1:  99.85  7142
             MISC: precision:  99.53%; recall:  99.42%; FB1:  99.48  3434
              ORG: precision:  99.78%; recall:  99.67%; FB1:  99.72  6314
              PER: precision:  99.86%; recall:  99.89%; FB1:  99.88  6602
processed 54612 tokens with 5942 phrases; found: 5957 phrases; correct: 5605.
accuracy:  99.08%; precision:  94.09%; recall:  94.33%; FB1:  94.21
              LOC: precision:  95.91%; recall:  97.11%; FB1:  96.51  1860
             MISC: precision:  90.01%; recall:  87.96%; FB1:  88.97  901
              ORG: precision:  92.92%; recall:  90.08%; FB1:  91.48  1300
              PER: precision:  95.04%; recall:  97.83%; FB1:  96.42  1896
processed 49888 tokens with 5648 phrases; found: 5662 phrases; correct: 5149.
accuracy:  98.25%; precision:  90.94%; recall:  91.17%; FB1:  91.05
              LOC: precision:  91.61%; recall:  93.59%; FB1:  92.59  1704
             MISC: precision:  81.46%; recall:  79.49%; FB1:  80.46  685
              ORG: precision:  89.30%; recall:  88.44%; FB1:  88.87  1645
              PER: precision:  95.88%; recall:  96.54%; FB1:  96.21  1628                            
```                            